# **CSC173: Group Activity 1**
>*Members: Febe Belvis, Kervin Paalisbo, Joshua Radz Adlaon*

This notebook documents this group's Activity 1, where the goal is to implement a neural network from scratch (without using machine learning libraries such as TensorFlow or PyTorch). The neural network will be trained and evaluated on the Breast Cancer dataset from the UCI Machine Learning Repository.

***

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo # to directly import the dataset from its repo


***

## Loading and Preprocessing the Dataset

Before we use train the network, we import the dataset from its repository using the ```ucimlrepo``` library. 

In [25]:
breast_cancer = fetch_ucirepo(id=17) # import the dataset from the ML repo

# extract features and its values/targets
X = breast_cancer.data.features
y = breast_cancer.data.targets

# combine into one DataFrame
df = pd.concat([X, y], axis=1)

We then preprocess the dataset by standardizing the feature names and converting the diagnosis values from ```'M/B'``` to ```'1/0'```.

In [26]:
# lowercase 'diagnosis' column for standardization 
df.rename(columns={df.columns[-1]: 'diagnosis'}, inplace=True)

# convert diagnosis values to binary (M=1, B=0)
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

df.head(25)

,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,fractal_dimension1,...,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3,diagnosis
0,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.71190,0.26540,0.4601,0.11890,1
1,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.24160,0.18600,0.2750,0.08902,1
2,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.45040,0.24300,0.3613,0.08758,1
3,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.68690,0.25750,0.6638,0.17300,1
4,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.40000,0.16250,0.2364,0.07678,1
5,12.450,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.5249,0.53550,0.17410,0.3985,0.12440,1
6,18.250,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,...,27.66,153.20,1606.0,0.1442,0.2576,0.37840,0.19320,0.3063,0.08368,1
7,13.710,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,...,28.14,110.60,897.0,0.1654,0.3682,0.26780,0.15560,0.3196,0.11510,1
8,13.000,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389,...,30.73,106.20,739.3,0.1703,0.5401,0.53900,0.20600,0.4378,0.10720,1
9,12.460,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243,...,40.68,97.65,711.4,0.1853,1.0580,1.10500,0.22100,0.4366,0.20750,1


***
## Feature Selection (using Correlation)
In this step, we identify the two features most correlated with the target ```diagnosis```.

We use the **Pearson correlation coefficient**, which measures the linear relationship between two variables. 


In [31]:
# compute correlation with the target 'diagnosis'
corr = df.corr()['diagnosis'].abs().sort_values(ascending=False)

# display top correlations
corr.head(10)

diagnosis          1.000000
concave_points3    0.793566
perimeter3         0.782914
concave_points1    0.776614
radius3            0.776454
perimeter1         0.742636
area3              0.733825
radius1            0.730029
area1              0.708984
concavity1         0.696360
Name: diagnosis, dtype: float64

*A higher absolute correlation value (closer to 1 or -1) means that changes in the feature are strongly associated with changes in the target, making that feature more useful for prediction.*

<span style="font-size:12px">*Source: GeeksforGeeks. (2025, July 23). Pearson Correlation Coefficient. GeeksforGeeks. https://www.geeksforgeeks.org/maths/pearson-correlation-coefficient/*</span>




<br><br>Since we're limited to using only 2 input features, we pick the top 2 most correlated features to ```diagnosis```

In [ ]:
top_features = corr.index[1:3].tolist()

# extract top 2 correlated features
X_selected = X[top_features]

print("Selected features:", top_features)

Selected features: ['concave_points3', 'perimeter3']
     concave_points3  perimeter3
0             0.2654      184.60
1             0.1860      158.80
2             0.2430      152.50
3             0.2575       98.87
4             0.1625      152.20
..               ...         ...
564           0.2216      166.10
565           0.1628      155.00
566           0.1418      126.70
567           0.2650      184.60
568           0.0000       59.16

[569 rows x 2 columns]
